In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [8]:
# Define training data
input_texts = ['hi', 'how are you?', 'what is your name?', 'bye','what is your favorite color?']
target_texts = ['hello', 'I am fine, thank you.', 'I am a chatbot.', 'goodbye','blue']

# Tokenization
input_characters = sorted(list(set(' '.join(input_texts))))
target_characters = sorted(list(set(' '.join(target_texts))) + ['\t'])  # Include '\t' in target characters

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# Create encoder and decoder data
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

# Define model
latent_dim = 256

encoder_inputs = tf.keras.layers.Input(shape=(None, num_encoder_tokens))
encoder = tf.keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.layers.Input(shape=(None, num_decoder_tokens))
decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

#early_stopping=tf.keras.callbacks.EarlyStopping()

# Compile and train the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=1, epochs=1000, validation_split=0.2)

# Define inference models
encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

decoder_state_input_h = tf.keras.layers.Input(shape=(latent_dim,))
decoder_state_input_c = tf.keras.layers.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Function to generate responses
def respond(input_text):
    # Encode the input text
    input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    for t, char in enumerate(input_text):
        input_seq[0, t, input_token_index[char]] = 1.0

    # Get the initial decoder state from the encoder
    states_value = encoder_model.predict(input_seq)

    # Initialize the target sequence with a start character
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0

    # Decode sequence one character at a time
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_characters[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]

    return decoded_sentence

Epoch 1/1000
4/4 [==============================] - 2s 114ms/step - loss: 1.6152 - accuracy: 0.0119 - val_loss: 0.4452 - val_accuracy: 0.0000e+00
Epoch 2/1000
4/4 [==============================] - 0s 19ms/step - loss: 1.7223 - accuracy: 0.0952 - val_loss: 0.4221 - val_accuracy: 0.0000e+00
Epoch 3/1000
4/4 [==============================] - 0s 19ms/step - loss: 1.4727 - accuracy: 0.0476 - val_loss: 0.4449 - val_accuracy: 0.0000e+00
Epoch 4/1000
4/4 [==============================] - 0s 18ms/step - loss: 1.4132 - accuracy: 0.0714 - val_loss: 0.4415 - val_accuracy: 0.0000e+00
Epoch 5/1000
4/4 [==============================] - 0s 18ms/step - loss: 1.4058 - accuracy: 0.0595 - val_loss: 0.4789 - val_accuracy: 0.0000e+00
Epoch 6/1000
4/4 [==============================] - 0s 18ms/step - loss: 1.3613 - accuracy: 0.0714 - val_loss: 0.5056 - val_accuracy: 0.0000e+00
Epoch 7/1000
4/4 [==============================] - 0s 19ms/step - loss: 1.3293 - accuracy: 0.0833 - val_loss: 0.4937 - val_accur

In [9]:
print(respond('hi'))

1/1 [==============================] - 0s 10ms/step
ello..................
